In [1]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim


In [2]:
data_path = 'data/data.csv'

In [3]:
df = pd.read_csv(data_path)

In [4]:
print(f'training set shape : {df.shape}')
print(df.columns)

training set shape : (569, 33)
Index(['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst', 'Unnamed: 32'],
      dtype='object')


In [5]:
df = df.drop('Unnamed: 32', axis=1)
x = df.drop(['id', 'diagnosis'], axis=1)
y = df.iloc[:,1]



In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(x, y, shuffle=True, test_size=0.2)


In [7]:
x_train = torch.tensor(X_train.values.astype(np.float32), dtype=torch.float32, requires_grad=True)
Y_train = Y_train.replace('M', 0)
Y_train = Y_train.replace('B', 1)
y_train = torch.tensor(Y_train.values.astype(np.float32), dtype=torch.float, requires_grad=True)

x_test = torch.tensor(X_test.values.astype(np.float32), dtype=torch.float32, requires_grad=True)
Y_test = Y_test.replace('M', 0)
Y_test = Y_test.replace('B', 1)
y_test = torch.tensor(Y_test.values.astype(np.float32), dtype=torch.float32, requires_grad=True)

In [8]:
print(y_train)
print(x_train)

tensor([0., 1., 1., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1., 0.,
        1., 1., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1.,
        1., 1., 1., 0., 0., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1.,
        0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 1.,
        1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0., 1., 0., 1., 0., 1., 0., 1.,
        1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 0., 0., 1.,
        1., 0., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 0., 1., 0., 1., 1.,
        1., 1., 1., 0., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 0., 0.,
        1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0.,
        1., 1., 0., 0., 0., 1., 0., 1., 1., 0., 0., 1., 1., 1., 0., 1., 0., 1.,
        1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1.,
        1., 1., 1., 0., 0., 1., 1., 1., 

In [9]:
def generates_batches(x, y, batch_size):
    batches = []
    num_batches = int(x.shape[0]/batch_size)

    for i in range(num_batches):
        batches.append(
                     (x[i*batch_size: (i+1)*batch_size], 
                      y[i*batch_size: (i+1)*batch_size])
            )

    if(num_batches * batch_size < x.shape[0]):
        batches.append(
            (x[num_batches*batch_size : x.shape[0]],
                y[num_batches*batch_size : x.shape[0]])
            
        )
    return batches
    
    

In [10]:
train_set = generates_batches(x_train, y_train, 4)
print(train_set.__len__())


test_set = generates_batches(x_test, y_test, 4)


114


In [22]:

class feedforward(nn.Module):
    def __init__(self):
        super().__init__()

        self.hidden_layers = nn.Sequential(
            nn.Linear(30, 120),
            nn.ReLU(),
            nn.Linear(120, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        x = self.hidden_layers(x)
        x = x.view(-1)
        return x
    
    @property
    def model_name(self):
        return "feedforward"
        

In [38]:
from Trainer import trainer

model = feedforward()
loss_fn = nn.BCELoss()
optimizer = optimizer = optim.SGD(model.parameters(), lr=1e-4)
max_epochs = 100
model_trainer = trainer(model, optimizer, loss_fn, train_set, test_set, max_epochs, model_name = model.model_name , when_to_stop=4)



In [24]:
model_trainer.fit()

epoch : 1, train_loss : 4.8161 ,  val_loss : 3.9942
epoch : 2, train_loss : 0.3844 ,  val_loss : 0.3288
epoch : 3, train_loss : 0.3344 ,  val_loss : 0.2717
epoch : 4, train_loss : 0.2977 ,  val_loss : 0.2348
epoch : 5, train_loss : 0.2743 ,  val_loss : 0.2139
epoch : 6, train_loss : 0.2598 ,  val_loss : 0.2006
epoch : 7, train_loss : 0.2515 ,  val_loss : 0.1914
epoch : 8, train_loss : 0.2461 ,  val_loss : 0.1844
epoch : 9, train_loss : 0.2419 ,  val_loss : 0.1791
epoch : 10, train_loss : 0.2384 ,  val_loss : 0.1749
epoch : 11, train_loss : 0.2356 ,  val_loss : 0.1715
epoch : 12, train_loss : 0.2332 ,  val_loss : 0.1686
epoch : 13, train_loss : 0.2312 ,  val_loss : 0.1662
epoch : 14, train_loss : 0.2296 ,  val_loss : 0.1640
epoch : 15, train_loss : 0.2282 ,  val_loss : 0.1620
epoch : 16, train_loss : 0.2270 ,  val_loss : 0.1603
epoch : 17, train_loss : 0.2260 ,  val_loss : 0.1588
epoch : 18, train_loss : 0.2251 ,  val_loss : 0.1575
epoch : 19, train_loss : 0.2243 ,  val_loss : 0.1563
ep

In [41]:
def check_accuracy(x, y):
    y_pred = model(x)
    num_correct = 0
    for i in range(y_pred.shape[0]):
        if(abs(y_pred[i].item() - y[i]) < 0.2):
            num_correct += 1
    acc = num_correct/x.shape[0] * 100
    print(f'{acc:.2f}%' )

In [42]:

for batch in test_set:
    x, y = batch
    check_accuracy(x, y)

50.00%
0.00%
100.00%
0.00%
50.00%
25.00%
50.00%
0.00%
0.00%
50.00%
0.00%
75.00%
50.00%
25.00%
75.00%
25.00%
50.00%
50.00%
0.00%
50.00%
50.00%
25.00%
50.00%
50.00%
50.00%
50.00%
25.00%
75.00%
50.00%
